In [ ]:
import importlib

import ipynb.fs.full.preprocessing0
import ipynb.fs.full.preprocessing05
#import ipynb.fs.full.prepossessing075_akistage
import preprocessing1
#import ipynb.fs.full.preprocessing2_BT
import preprocessing2_BT

import ipynb.fs.full.preprocessing25_BTcorr
import ipynb.fs.full.preprocessing3_smote
#import ipynb.fs.full.preprocessing4
import preprocessing4

#import ipynb.fs.full.runxgboost
import runxgboost

#import ipynb.fs.full.postprocessing1_SHAP

import postprocessing1_SHAP

import ipynb.fs.full.postprocessing3_collect

from ipynb.fs.full.slackbot import ping_slack
import pandas as pd
import xgboost as xgb
import numpy as np
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTENC
import matplotlib.pyplot as plt
from PIL import Image
from scipy.interpolate import BSpline, make_interp_spline, interp1d
#import rpy2.rstrs as rstrs
#from rpy2.rstrs.packages import importr
import csv
from dfply import *
from xgboost import XGBClassifier
import itertools
import os
from tqdm import tqdm
from os import listdir
from os.path import isfile, join
from os.path import exists
import logging
import time
import pickle
import utils_function
from sklearn.metrics import roc_auc_score

import plot_utils

In [ ]:
#Reload library to update on any change
importlib.reload(ipynb.fs.full.preprocessing0)
importlib.reload(ipynb.fs.full.preprocessing05)
#importlib.reload(ipynb.fs.full.preapossessing075_akistage)
importlib.reload(preprocessing1)
importlib.reload(preprocessing2_BT)
importlib.reload(ipynb.fs.full.preprocessing25_BTcorr)
importlib.reload(ipynb.fs.full.preprocessing3_smote)
# #importlib.reload(ipynb.fs.full.preprocessing4)
importlib.reload(preprocessing4)
importlib.reload(runxgboost)
importlib.reload(postprocessing1_SHAP)
importlib.reload(ipynb.fs.full.postprocessing3_collect)
importlib.reload(postprocessing1_SHAP)
importlib.reload(utils_function)
importlib.reload(plot_utils)

In [ ]:
site_list = ['KUMC', 'UTSW', 'MCW', 'UofU', 'UIOWA', 'UMHC', 'UPITT', 'UTHSCSA', 'UNMC']
for site in site_list:
    # Read the config files for variables
    # The location of config files is at '/home/hoyinchan/code/AKI_CDM_PY/configs_files/publish_config/Configs_init.txt'
    configs_variables = utils_function.read_config(site, suffix='init', config_base_location='/home/hoyinchan/code/AKI_CDM_PY/configs_files/publish_config/')
    utils_function.write_config(configs_variables, suffix=site)
    configs_variables

In [ ]:
configs_variables = [utils_function.read_config(site) for site in site_list]

In [ ]:
configs_variables[7]['rerun_flag']=True

In [ ]:
#postprocessing1_SHAP.collectSHAPraw_cross_sub(configs_variables[7], configs_variables[7], returnflag=False)

In [ ]:
#ipynb.fs.full.postprocessing3_collect.collect_collectSHAPraw_cross_sub_pre(configs_variables)

In [ ]:
configs_variable = configs_variables[1]

In [ ]:
# #for configs_variable in configs_variables[[0,1,-1]]:
# for configs_variable in [configs_variables[i] for i in [1, 0, -1]]:
#     site, datafolder, home_directory = utils_function.get_commons(configs_variable)
#     print(site)
#     try:
#         dialysis, pdialysis, vdialysis = count_dialysis(configs_variable)
#         print(dialysis, pdialysis, vdialysis)
#     except:
#         pass

In [ ]:
def count_dialysis(configs_variable):
# configs_variable = configs_variables[1]
# if True:
    site, datafolder, home_directory = utils_function.get_commons(configs_variable)
    stg = configs_variable['stg']
    fs= configs_variable['fs']
    oversample=configs_variable['oversample']
    suffix='nc'    

    onset = pd.read_parquet(datafolder+site+'/p0_onset_'+site+'.parquet')
    covid = pd.read_parquet(datafolder+site+'/p0_covid_status_'+site+'.parquet')
    
    px = pd.read_parquet(datafolder+site+'/p0_px_'+site+'.parquet')
    px['PATID'] = px['PATID'].astype(str)
    px['ENCOUNTERID'] = px['ENCOUNTERID'].astype(str)        
    px['PX_TYPE'] = px['PX_TYPE'].astype(str)
    px = px.drop_duplicates()    

    # dx = pd.read_parquet(datafolder+site+'/p0_dx_current_'+site+'.parquet')
    lab = pd.read_parquet(datafolder+site+'/p0_lab_'+site+'.parquet')

    onset['PATID'] = onset['PATID'].astype(str)
    onset['ENCOUNTERID'] = onset['ENCOUNTERID'].astype(str)  
    covid['PATID'] = covid['PATID'].astype(str)
    covid['ENCOUNTERID'] = covid['ENCOUNTERID'].astype(str)  

    lab['PATID'] = lab['PATID'].astype(str)
    lab['ENCOUNTERID'] = lab['ENCOUNTERID'].astype(str)  

    px['PX'] = px['PX'].astype(str)

    dialysis_ch = ('90935','90937', '90940', '90945', '90947', '90989', '90993', '90997', '90999')
    dialysis_icd9 = ('39.93','39.95','54.98')
    dialysis_icd10 = ('5A1D00Z','5A1D60Z','5A1D70Z','5A1D80Z','5A1D90Z')

    dialysis = px[(px['PX_TYPE']=='CH') & (px['PX'].str.startswith(dialysis_ch, na=False)) | (px['PX_TYPE']=='10') & (px['PX'].str.startswith(dialysis_icd10, na=False)) | (px['PX_TYPE']=='09') & (px['PX'].str.startswith(dialysis_icd9, na=False))]

    dialysis_first = dialysis[['PATID', 'ONSETS_ENCOUNTERID', 'PX_DATE']].sort_values('PX_DATE').groupby(['PATID', 'ONSETS_ENCOUNTERID']).first().reset_index()
    dialysis_first['ONSETS_ENCOUNTERID'] = dialysis_first['ONSETS_ENCOUNTERID'].astype(str)   

    dialysis_first = dialysis_first.merge(covid, left_on=['PATID', 'ONSETS_ENCOUNTERID'], right_on=['PATID', 'ENCOUNTERID'], how='left')
    dialysis_first = dialysis_first[dialysis_first['BCCOVID']]
    dialysis_first = dialysis_first.drop(['BCCOVID'],axis=1)

    dialysis_first = dialysis_first.merge(onset, left_on=['PATID', 'ONSETS_ENCOUNTERID'], right_on=['PATID', 'ENCOUNTERID'], how='left')
    dialysis_first = dialysis_first[dialysis_first['AKI_STAGE']>0]
    dialysis_first = dialysis_first[['PATID', 'ONSETS_ENCOUNTERID', 'PX_DATE']]
            
    lablonic = {             "Potassium":('2823-3', '77142-8', '6298-4'), 
                             "pH":("11558-4", "97536-7", "2744-1", "2753-2"), 
                             "BUN":("59570-2", "6299-2", "3094-0", "14937-7", "12961-9")}

    lablonic_values = {code for values in lablonic.values() for code in values}

    filtered_lab = lab[lab['LAB_PX'].isin(lablonic_values)]

    filtered_lab = filtered_lab[filtered_lab['DAYS_SINCE_ADMIT']>=0]
    filtered_lab['ONSETS_ENCOUNTERID'] = filtered_lab['ONSETS_ENCOUNTERID'].astype(str)
    filtered_lab = dialysis_first.merge(filtered_lab, on=['PATID', 'ONSETS_ENCOUNTERID'], how='left')
    filtered_lab = filtered_lab[['PATID', 'ONSETS_ENCOUNTERID', 'PX_DATE', 'SPECIMEN_DATE', 'RESULT_NUM', 'LAB_PX']].groupby(['PATID', 'ONSETS_ENCOUNTERID', 'PX_DATE', 'SPECIMEN_DATE', 'LAB_PX']).mean().reset_index()
    filtered_lab = filtered_lab[filtered_lab['PX_DATE']>=filtered_lab['SPECIMEN_DATE']]
    filtered_lab = filtered_lab.sort_values('SPECIMEN_DATE', ascending=False).groupby(['PATID', 'ONSETS_ENCOUNTERID', 'LAB_PX']).first().reset_index()

    threshold = {'Potassium':6, 'pH':7.5, 'BUN':100.8}

    def map_lab_name(lab_px):
        for lab_name, codes in lablonic.items():
            if lab_px in codes:
                return lab_name
        return None

    filtered_lab = dialysis_first[['PATID', 'ONSETS_ENCOUNTERID']].drop_duplicates().merge(filtered_lab, on = ['PATID', 'ONSETS_ENCOUNTERID'], how='inner')
    filtered_lab['LAB_NAME'] = filtered_lab['LAB_PX'].apply(map_lab_name)
    filtered_lab = filtered_lab.dropna(subset=['LAB_NAME'])
    
    pivoted_df = filtered_lab.pivot(index=['PATID', 'ONSETS_ENCOUNTERID'], columns='LAB_NAME', values='RESULT_NUM').reset_index()    
    pivoted_df['hyperkalemia'] = pivoted_df['Potassium']>6
    pivoted_df['acidosis'] = pivoted_df['pH']<7.15
    pivoted_df['uremia'] = pivoted_df['BUN']>=100.8
    
    pivoted_df = dialysis_first.merge(pivoted_df, on = ['PATID', 'ONSETS_ENCOUNTERID'], how='inner')
        
    return dialysis_first, pivoted_df
        
#     for key in ['Potassium', 'pH', 'BUN']:
#         filtered_lab2 = filtered_lab[filtered_lab['LAB_PX'].isin(lablonic[key])]
#         if key == 'Potassium':
#             filtered_lab2 = filtered_lab2[filtered_lab2['RESULT_NUM']<6]
#         elif key == 'pH':
#             filtered_lab2 = filtered_lab2[filtered_lab2['RESULT_NUM']>7.15]        
#         else:
#             filtered_lab2 = filtered_lab2[filtered_lab2['RESULT_NUM']<=100.8]
            
                
#         dialysis_first = dialysis_first.merge(filtered_lab2[['PATID', 'ONSETS_ENCOUNTERID']], on = ['PATID', 'ONSETS_ENCOUNTERID'], how='inner')
    
#     pdialysis = dialysis_first[['PATID', 'ONSETS_ENCOUNTERID']].drop_duplicates().shape[0]
       
#     return ndialysis, pdialysis, vdialysis

In [ ]:
configs_variable

In [ ]:
dialysis_first, dialysis_condition_lab = count_dialysis(configs_variable)

In [ ]:
dialysis_first

In [ ]:
dialysis_condition_lab

In [ ]:
site, datafolder, home_directory = utils_function.get_commons(configs_variables[1])
stg = configs_variable['stg']
fs= configs_variable['fs']
oversample=configs_variable['oversample']
suffix='nc'    

dx = pd.read_parquet(datafolder+site+'/p0_dx_current_'+site+'.parquet')
dx['PATID'] = dx['PATID'].astype(str)
dx['ENCOUNTERID'] = dx['ENCOUNTERID'].astype(str)        
dx['DX_TYPE'] = dx['DX_TYPE'].astype(str)
dx = dx.drop_duplicates()    

In [ ]:
dx_dx9_range  = {'Pulmonary edema':('514'), 
                 'Oliguria or anuria':('788.5')
                  }              
dx_dx10_range  = {'Pulmonary edema':('J81.0','J81.1'), 
                  'Oliguria or anuria':('R34')
                  }        
dx9_values = {code for values in dx_dx9_range.values() for code in values}
dx10_values = {code for values in dx_dx10_range.values() for code in values}

dx['DX_TYPE'] = dx['DX_TYPE'].astype(str)

dx = dx[((dx['DX_TYPE']=='10') &  dx['DX'].isin(dx10_values)) | ((dx['DX_TYPE']!='10') &  dx['DX'].isin(dx9_values))]

dx['ONSETS_ENCOUNTERID'] = dx['ONSETS_ENCOUNTERID'].astype(str)

In [ ]:
filtered_dx = dialysis_first[['PATID', 'ONSETS_ENCOUNTERID']].merge(dx, on = ['PATID', 'ONSETS_ENCOUNTERID'], how='left', indicator=True)

filtered_dx = filtered_dx[filtered_dx['_merge']=='left_only']

In [ ]:
filtered_dx

In [ ]:
dx

In [ ]:
dialysis_first2